In [1]:
from collections import ChainMap

In [2]:
from k8stencil import patterns

In [3]:
pod_affinity_labels =  {
    "app.kubernetes.io/name": "thanos-receive",
    "app.kubernetes.io/instance": "thanos-receive-ingestor-default",
}

In [4]:
match_labels = ChainMap(
    {
        "app.kubernetes.io/component": "database-write-hashring",
        "controller.receive.thanos.io/hashring": "default",
    },
    pod_affinity_labels,
)

In [5]:
spec_metadata_labels = ChainMap(
    {
        "app.kubernetes.io/version": "v0.31.0" ,
    },
    match_labels,
)

In [6]:
metadata_labels = ChainMap(
    {
        "controller.receive.thanos.io": "thanos-receive-controller",
    },
    spec_metadata_labels,
)

In [7]:
ports = dict(
    grpc=10901,
    http=10902,
    remote_write=19291,
)

In [9]:
args = dict(
    log=dict(
        level="info",
        format="logfmt",
    ),
    grpc_address=patterns.address(ports, "grpc"),
    http_address=patterns.address(ports, "http"),
    remote_write=dict(
        address=patterns.address(ports, "remote_write"),
    ),
    receive=dict(
        replication_factor=1,
        local_endpoint="$(NAME).thanos-receive-ingestor-default.$(NAMESPACE).svc.cluster.local:10901",
        hashrings_file="/var/lib/thanos-hashrings.json"
    ),
    label=[
        'replica="$(NAME)"',
        'receive="true"',
    ],
    tsdb=dict(
        path="/var/thanos/receive",
        retention="15d",
    ),
)

In [10]:
env_fields = dict(
    name="metadata.name",
    namespace="metadata.namespace",
    host_ip_address="status.hostIP",
)

In [11]:
mounts={
    "/var/thanos/receive": dict(name="data", readOnly=False),
    "/var/lib/thanos-receive": dict(name="hashring-config"),
}

In [12]:
containers = [
    dict(
        args=list(patterns.args_to_list(args)),
        env=list(patterns.env_fields_to_env(env_fields)),
        ports=list(patterns.ports_to_container_ports(ports)),
        readinessProbe=patterns.http_probe(ports, 20, 5, "ready"),
        livenessProbe=patterns.http_probe(ports, 30, 8, "healthy"),
        volumeMounts=list(patterns.get_volume_mounts(mounts)),
        image="quay.io/thanos/thanos:v0.31.0",
        name="thanos-receive",
        resources={},
        terminationMessagePolicy="FallbackToLogsOnError",
    ),
]

In [23]:
statefulset_spec_template_spec = dict(
    affinity=get_pod_anti_affinitypatterns(pod_affinity_labels, "thanos"),
    nodeSelector={"kubernetes.io/os": "linux"},
    securityContext={
        'fsGroup': 65534,
        'runAsGroup': 65532,
        'runAsNonRoot': True,
        'runAsUser': 65534,
        'seccompProfile': {'type': 'RuntimeDefault'}
    },
    containers=containers,
)

In [24]:
statefulset_spec = dict(
  minReadySeconds=0,
  replicas=1,
  selector=dict(
      matchLabels=match_labels,
  ),
  serviceName="thanos-receive-ingestor-default",
  template=dict(
    metadata=dict(labels=spec_metadata_labels),
    spec=statefulset_spec_template_spec,
    volumeClaimTemplates=dict(
        dict(
            metadata=dict(
                labels=match_labels,
                name="data",
            ),
            spec=dict(
                accessModes=["ReadWriteOnce"],
                resources=dict(requests=dict(storage="10Gi")),
            ),
        ),
    ),
  ),
)

In [22]:
statefulset = dict(
    apiVersion="apps/v1",
    kind="StatefulSet",
    metadata=dict(
        labels=metadata_labels,
        name=metadata_labels["app.kubernetes.io/instance"],
        namespace="thanos",
    ),
    spec=statefulset_spec,
)